In [17]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-70b-versatile",
    groq_api_key='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',
)
response = llm.invoke("The first person to land on the moon was ...")
print(response.content)


Neil Armstrong. He stepped onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-31227?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print (page_data)

Apply for Principal Software Engineer - Android Mobile Platform - Remote Work Option

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyPrincipal Software Engineer - Android Mobi

In [20]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ###INSTRUCTION:
    The scraped text is from the carrier's page of a website.
    Your job is to extract the job postings and return then in JSON format containing 
    the following keys: 'role','expirence','skills','description'.
    Only return the valid JSON.
    ###VALID JSON (NO PREAMBLE):{page_data}
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Principal Software Engineer - Android Mobile Platform",
  "experience": "Minimum 12+ years of experience in software engineering, with a focus on Android mobile engineering",
  "skills": [
    "Android/Kotlin development",
    "Agile development methodologies",
    "Scrum framework",
    "DevOps practices",
    "Continuous integration and delivery (CI/CD)",
    "Strong technical leadership skills",
    "Innovative problem-solving skills",
    "Excellent communication and collaboration skills"
  ],
  "description": "We are seeking a Principal Engineer to drive our Android Mobile platform at Nike. Your expertise in Android/Kotlin development will help us stay ahead of the curve in this constantly evolving landscape and chip in significantly to our success! The ideal candidate will be a servant leader, bringing deep technical expertise to deliver outstanding agility, availability, resiliency, security, and performant mobile solutions."
}
```


In [22]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Principal Software Engineer - Android Mobile Platform',
 'experience': 'Minimum 12+ years of experience in software engineering, with a focus on Android mobile engineering',
 'skills': ['Android/Kotlin development',
  'Agile development methodologies',
  'Scrum framework',
  'DevOps practices',
  'Continuous integration and delivery (CI/CD)',
  'Strong technical leadership skills',
  'Innovative problem-solving skills',
  'Excellent communication and collaboration skills'],
 'description': 'We are seeking a Principal Engineer to drive our Android Mobile platform at Nike. Your expertise in Android/Kotlin development will help us stay ahead of the curve in this constantly evolving landscape and chip in significantly to our success! The ideal candidate will be a servant leader, bringing deep technical expertise to deliver outstanding agility, availability, resiliency, security, and performant mobile solutions.'}

In [23]:
type(json_res)

dict

In [24]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [26]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [32]:
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [29]:
job = json_res
job['skills']

['Android/Kotlin development',
 'Agile development methodologies',
 'Scrum framework',
 'DevOps practices',
 'Continuous integration and delivery (CI/CD)',
 'Strong technical leadership skills',
 'Innovative problem-solving skills',
 'Excellent communication and collaboration skills']

In [34]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sushant, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Android Mobile Platform Solutions for Nike

Dear Hiring Manager,

I came across the job description for a Principal Software Engineer - Android Mobile Platform at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your goals.

With over a decade of experience in software engineering, our team of experts specializes in Android/Kotlin development, Agile development methodologies, Scrum framework, DevOps practices, and Continuous integration and delivery (CI/CD). Our strong technical leadership skills, innovative problem-solving skills, and excellent communication and collaboration skills make us an ideal partner for driving your Android Mobile platform forward.

Our portfolio showcases our expertise in Android/Kotlin development, with successful projects such as [https://example.com/kotlin-android-portfolio](https://example.com/kotlin-android-portfolio) and [https://example.com/android-tv-portfolio](https://example.com/